In [2]:
import rospy
from std_msgs.msg import Int32
from cv_bridge import CvBridge
import cv2
import os
import copy
import argparse
import itertools
from collections import Counter
from collections import deque
from matplotlib import pyplot as plt
import time
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy import stats
import tensorflow as tf

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# Actions that we try to detect
actions = np.array(['follow', 'stop', 'turn_left', 'turn_right', 'move_forward', 'move_backward'])

model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.load_weights('action.h5')

'''colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        #cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        #cv2.rectangle(output_frame,(384,0),(510,128),(0,255,0),-1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame'''

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    max_prob = 0
    max_prob_index = -1
    for num, prob in enumerate(res):
        if prob > max_prob:
            max_prob = prob
            max_prob_index = num

    cv2.putText(output_frame, actions[max_prob_index], (0, 85+max_prob_index*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame, max_prob_index

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z]) 
            
    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list)) 

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list)) 
    
    return temp_landmark_list

def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)

    return image

# Initialize ROS node
rospy.init_node('action_detection_node', anonymous=True)

# Create publisher for action topic
action_publisher = rospy.Publisher('action', Int32, queue_size=10)

# Create CvBridge object
bridge = CvBridge()

# For webcam input:
cap = cv2.VideoCapture(3)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
use_brect = True

with mp_hands.Hands(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
      while cap.isOpened():
        success, image = cap.read()
        debug_image = copy.deepcopy(image)
        if not success:
              print("Ignoring empty camera frame.")
              # If loading a video, use 'break' instead of 'continue'.
              continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        
        handedness = results.multi_handedness
        print(results)
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                               mp_drawing_styles.get_default_hand_landmarks_style(), 
                                               mp_drawing_styles.get_default_hand_connections_style())
                    # Bounding box calculation
                    brect = calc_bounding_rect(debug_image, hand_landmarks)
                    
                    # Landmark calculation
                    landmark_list = calc_landmark_list(debug_image, hand_landmarks)

                    # Conversion to relative coordinates / normalized coordinates
                    pre_processed_landmark_list = pre_process_landmark(landmark_list)
                    
                    if len(pre_processed_landmark_list)==126: 
                        continue
                    elif len(pre_processed_landmark_list)==63:
                        pre_processed_landmark_list.extend(np.zeros(21*3))
                    else:
                        pre_processed_landmark_list.extend(np.zeros(21*6))
                    
                    # Drawing part
                    image = draw_bounding_rect(use_brect, image, brect)    
        
        # 2. Prediction logic
        sequence.append(pre_processed_landmark_list)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (1265, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (250, 235, 245), 2, cv2.LINE_AA)
        
        
        #ROS action publish
        
        # Set the loop rate
        rate = rospy.Rate(1) # 1 Hz

        # Publish the actions in the list
        action = actions[max_prob_index]
        rospy.loginfo('Publishing action: %s', action)
        pub.publish(action)
        rate.sleep()
        
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', image) #cv2.flip(image, 1))
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

if __name__ == '__main__':
    try:
        action_publisher()
    except rospy.ROSInterruptException:
        pass

ModuleNotFoundError: No module named 'rospy'

In [63]:
results.multi_hand_landmarks

[landmark {
   x: 0.8205653429031372
   y: 0.4372186064720154
   z: 2.3218386502321664e-07
 }
 landmark {
   x: 0.7993267774581909
   y: 0.43032246828079224
   z: -0.017406750470399857
 }
 landmark {
   x: 0.7776250243186951
   y: 0.40702715516090393
   z: -0.026051992550492287
 }
 landmark {
   x: 0.7651407718658447
   y: 0.3742293119430542
   z: -0.032751407474279404
 }
 landmark {
   x: 0.7604789137840271
   y: 0.3401475250720978
   z: -0.0382261760532856
 }
 landmark {
   x: 0.7699438333511353
   y: 0.3386610448360443
   z: -0.011902453377842903
 }
 landmark {
   x: 0.7529102563858032
   y: 0.29939010739326477
   z: -0.023005390539765358
 }
 landmark {
   x: 0.7485141754150391
   y: 0.2718585133552551
   z: -0.032092660665512085
 }
 landmark {
   x: 0.7488373517990112
   y: 0.24528968334197998
   z: -0.03840484470129013
 }
 landmark {
   x: 0.7826817035675049
   y: 0.3206998109817505
   z: -0.009550678543746471
 }
 landmark {
   x: 0.7675331234931946
   y: 0.2729939818382263
   z: 

In [64]:
len(results.multi_hand_landmarks)

1

In [68]:
len(pre_processed_landmark_list)

126

In [59]:
cap.release()
cv2.destroyAllWindows()

In [69]:
pre_processed_landmark_list

[0.0,
 0.0,
 9.47689244992721e-10,
 -0.1673469387755102,
 -0.0326530612244898,
 -7.104796110367288e-05,
 -0.3346938775510204,
 -0.1346938775510204,
 -0.00010633466347139709,
 -0.4326530612244898,
 -0.27755102040816326,
 -0.00013367921418073225,
 -0.46938775510204084,
 -0.42857142857142855,
 -0.00015602520838075754,
 -0.39591836734693875,
 -0.43673469387755104,
 -4.8581442358542464e-05,
 -0.5306122448979592,
 -0.6081632653061224,
 -9.389955322353208e-05,
 -0.563265306122449,
 -0.7306122448979592,
 -0.0001309904516959677,
 -0.563265306122449,
 -0.8489795918367347,
 -0.00015675446816853115,
 -0.2979591836734694,
 -0.5142857142857142,
 -3.898236140304682e-05,
 -0.4163265306122449,
 -0.726530612244898,
 -7.630835990516507e-05,
 -0.43673469387755104,
 -0.8693877551020408,
 -0.00010712019795057725,
 -0.42448979591836733,
 -1.0,
 -0.00012947086777005876,
 -0.17959183673469387,
 -0.5428571428571428,
 -4.007855848390229e-05,
 -0.24897959183673468,
 -0.746938775510204,
 -8.239750655329957e-05,
 -